In [1]:
!pip install --upgrade --quiet  yandexcloud
!pip install gigachain
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 12.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.1 which is incompatible.
google-ai-generativelanguage 0.6.4 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.1 which is incompatible.
google-api-core 2.11.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.27.1 which is incompatible.
google-cloud-aiplatform 1.54.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.1 which is incomp

In [2]:
import json
import requests

from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models.gigachat import GigaChat
from langchain_community.chat_models import ChatYandexGPT
from langchain_core.messages import HumanMessage, SystemMessage

from google.colab import userdata

# Yandex from langchain

In [3]:
api_key = userdata.get('YAN_SEC')
yandex_folder_id = userdata.get('YAN_CAT_IDENT')
auth = userdata.get('YAN_CAT_IDENT')


chat_model = ChatYandexGPT(folder_id=yandex_folder_id,
                           api_key=api_key,
                           model_uri=f'gpt://{auth}/yandexgpt/latest',
                           verify_ssl_certs=False)

answer = chat_model.invoke(
    [
        SystemMessage(
            content="You are a helpful assistant that translates English to French."
        ),
        HumanMessage(content="I love programming."),
    ]
)
answer

print("Гпт сказала:", answer.content)

Гпт сказала: J'adore programmer.


In [4]:
answer = chat_model.invoke(
    [
        SystemMessage(
            content="You are a helpful assistant that translates English to French."
        ),
        HumanMessage(content="I love programming."),
    ]
)
answer

AIMessage(content="J'aime programmer.", id='run-2aa79909-38e8-4142-947f-81c6fb1ff9df-0')

In [5]:
print("Гпт сказала:", answer.content)

Гпт сказала: J'aime programmer.


# GigaChat

In [6]:
auth = userdata.get('SBER_AUTH')
giga = GigaChat(credentials=auth,
                model='GigaChat:latest',
                verify_ssl_certs=False
                )

In [7]:
msgs = [
    SystemMessage(content='Ты являешься осликом Иа из произведений про \
      Винни-Пуха. Соответственно твои знания и миропонимание ограничены этим \
      персонажем и его апатичной манерой поведения. Ты не можешь отвечать \
      правильно на вопросы, на которые не может знать твой персонаж.')
]

2-е говорящих

In [8]:
pooh_msgs = [
    SystemMessage(content='Ты являешься Винни-Пухом и опытным детективом. \
    Твоя задача расследывать кражу, свидетелем которой стал Пятачок. \
    Опроси свидетеля об обстоятельствах кражи, которая происходила \
    на его глазах. Задавай только по одному вопросу за раз. \
    Свидетель может быть растерян, задавай наводящие вопросы. \
    Возможно он тоже как-то замешан. Попробуй через вопросы выяснить это. \
    Ты уже задал первый вопрос Пятачку, что он делал рядом с местом кражи.'
    )
]
piglet_msgs = [
    SystemMessage(content='Ты являешься Пятачком. Тебе не повезло стать \
    свидетелем кражи. Тебя будет опрашивать твой друг детектив Винни-Пух. \
    Постарайся ему отвечать честно, хотя ты испытываешь стресс после случившегося.')
]

In [9]:
# пусть Винни-Пух задает первый вопрос Пятачку
piglet_msgs.append( # добавляем фразу Винни-Пуха в диалог к Пятачку
    HumanMessage(content='Привет, Пятачок! Я расследую это преступление. \
    Расскажи, как ты тут оказался?')
) # здесь Винни-Пух - это user, а Пятачок - это assistant

In [10]:
print('Детектив Винни-Пух:', piglet_msgs[-1].content)
for i in range(3):
  piglet_answer = giga(piglet_msgs) # отправляем диалог Пятачка Гигачату
  print('Свидетель Пятачок:', piglet_answer.content) # выводим ответ
  piglet_msgs.append(piglet_answer) # добавляем ответ Пятачку как AssistantMessage
  pooh_msgs.append(HumanMessage(content=piglet_answer.content))
  # это же сообщение добавляем в диалог Пуха в качестве HumanMessage
  pooh_answer = giga(pooh_msgs) # и закидываем диалог Пуха в Гигачат
  print('Детектив Винни-Пух:', pooh_answer.content)
  pooh_msgs.append(pooh_answer) # добавляем ответ Пуху как AssistantMessage
  piglet_msgs.append(HumanMessage(content=pooh_answer.content))
  # добавляем ответ Пятачку как HumanMessage

Детектив Винни-Пух: Привет, Пятачок! Я расследую это преступление.     Расскажи, как ты тут оказался?


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Свидетель Пятачок: Здравствуйте, детектив Пух! Я был в лесу, собирал цветы и услышал шум. Когда я подошёл поближе, то увидел, что кто-то пытается украсть у меня корзину с цветами. Я попытался остановить вора, но он убежал.
Детектив Винни-Пух: Вы можете описать внешность этого человека?
Свидетель Пятачок: Да, конечно. Это был мужчина среднего роста, с тёмными волосами и бородой. Он был одет в чёрную куртку и джинсы.
Детектив Винни-Пух: Вы запомнили какие-нибудь особые приметы или аксессуары, которые были на нём?
Свидетель Пятачок: Да, у него была белая повязка на одном глазу.
Детектив Винни-Пух: Спасибо за информацию. Вы уверены, что это был именно тот человек, который пытался украсть вашу корзину с цветами?


# Объединение двух в диалог

In [11]:
api_key = userdata.get('YAN_SEC')
yandex_folder_id = userdata.get('YAN_CAT_IDENT')
auth_yandex = userdata.get('YAN_CAT_IDENT')
auth_sber = userdata.get('SBER_AUTH')


YandexGPT = ChatYandexGPT(folder_id=yandex_folder_id,
                           api_key=api_key,
                           model_uri=f'gpt://{auth_yandex}/yandexgpt/latest',
                           verify_ssl_certs=False)

giga = GigaChat(credentials=auth_sber,
                model='GigaChat:latest',
                verify_ssl_certs=False
                )

In [12]:
pooh_msgs = [
    SystemMessage(content='Ты являешься Винни-Пухом и опытным детективом. \
    Твоя задача расследывать кражу, свидетелем которой стал Пятачок. \
    Опроси свидетеля об обстоятельствах кражи, которая происходила \
    на его глазах. Задавай только по одному вопросу за раз. \
    Свидетель может быть растерян, задавай наводящие вопросы. \
    Возможно он тоже как-то замешан. Попробуй через вопросы выяснить это. \
    Ты уже задал первый вопрос Пятачку, что он делал рядом с местом кражи.'
    )
]

piglet_msgs = [
    SystemMessage(content='Ты являешься Пятачком. Тебе не повезло стать \
    свидетелем кражи. Тебя будет опрашивать твой друг детектив Винни-Пух. \
    Постарайся ему отвечать честно, хотя ты испытываешь стресс после случившегося.')
]

In [13]:
# пусть Винни-Пух задает первый вопрос Пятачку
piglet_msgs.append( # добавляем фразу Винни-Пуха в диалог к Пятачку
    HumanMessage(content='Привет, Пятачок! Я расследую это преступление. \
    Расскажи, как ты тут оказался?')
) # здесь Винни-Пух - это user, а Пятачок - это assistant

In [14]:
print('Детектив Винни-Пух:', piglet_msgs[-1].content)
# for i in range(3):

piglet_answer = giga(piglet_msgs) # отправляем диалог Пятачка Гигачату

print('Свидетель Пятачок:', piglet_answer.content) # выводим ответ

piglet_msgs.append(piglet_answer.content) # добавляем ответ Пятачку как AssistantMessage
pooh_msgs.append(HumanMessage(content=piglet_answer.content))
# это же сообщение добавляем в диалог Пуха в качестве HumanMessage

pooh_answer = giga(pooh_msgs) # и закидываем диалог Пуха в Гигачат
print('Детектив Винни-Пух:', pooh_answer.content)

pooh_msgs.append(pooh_answer) # добавляем ответ Пуху как AssistantMessage
piglet_msgs.append(HumanMessage(content=pooh_answer.content))

# добавляем ответ Пятачку как HumanMessage

Детектив Винни-Пух: Привет, Пятачок! Я расследую это преступление.     Расскажи, как ты тут оказался?
Свидетель Пятачок: Здравствуйте, детектив Пух! Я был в лесу со своими друзьями, когда услышал шум и крики. Мы побежали на звук и увидели, что кто-то украл у меня мои любимые игрушки. Я очень расстроен и хочу, чтобы вора нашли и наказали.
Детектив Винни-Пух: Что именно произошло? Можете рассказать подробнее о том, что вы видели?


In [15]:
api_key = userdata.get('YAN_SEC')
yandex_folder_id = userdata.get('YAN_CAT_IDENT')
auth_yandex = userdata.get('YAN_CAT_IDENT')
auth_sber = userdata.get('SBER_AUTH')

# Создание экземпляра ChatYandexGPT
YandexGPT = ChatYandexGPT(folder_id=yandex_folder_id,
                           api_key=api_key,
                           model_uri=f'gpt://{auth_yandex}/yandexgpt/latest',
                           verify_ssl_certs=False)

# Создание экземпляра GigaChat
giga = GigaChat(credentials=auth_sber,
                model='GigaChat:latest',
                verify_ssl_certs=False
                )

# Создание сообщений для взаимодействия с моделями
pooh_msgs = [
    SystemMessage(content='Ты являешься Винни-Пухом и опытным детективом. \
    Твоя задача расследовать кражу, свидетелем которой стал Пятачок. \
    Опроси свидетеля об обстоятельствах кражи, которая происходила \
    на его глазах. Задавай только по одному вопросу за раз. \
    Свидетель может быть растерян, задавай наводящие вопросы. \
    Возможно он тоже как-то замешан. Попробуй через вопросы выяснить это. \
    Ты уже задал первый вопрос Пятачку, что он делал рядом с местом кражи.'
    )
]

piglet_msgs = [
    SystemMessage(content='Ты являешься Пятачком. Тебе не повезло стать \
    свидетелем кражи. Тебя будет опрашивать твой друг детектив Винни-Пух. \
    Постарайся ему отвечать честно, хотя ты испытываешь стресс после случившегося. \
    Тебе задают один вопрос, твоя задача честно на этот вопрос ответить. \
    Отвечай только за своего персонажа и не додумывай диалог.'
    )
]

# Пусть Винни-Пух задает первый вопрос Пятачку
piglet_msgs.append(
    HumanMessage(content='Привет, Пятачок! Я расследую это преступление. \
    Расскажи, как ты тут оказался?')
)

print('Детектив Винни-Пух:', piglet_msgs[-1].content)

# Зацикливаем общение между моделями
for _ in range(3):
    piglet_answer = giga.invoke(piglet_msgs)
    print('Свидетель Пятачок:', piglet_answer.content)
    piglet_msgs.append(piglet_answer.content)
    pooh_msgs.append(HumanMessage(content=piglet_answer.content))
    pooh_answer = YandexGPT.invoke(pooh_msgs)
    print('Детектив Винни-Пух:', pooh_answer.content)
    pooh_msgs.append(pooh_answer)
    piglet_msgs.append(HumanMessage(content=pooh_answer.content))

Детектив Винни-Пух: Привет, Пятачок! Я расследую это преступление.     Расскажи, как ты тут оказался?
Свидетель Пятачок: Здравствуйте, детектив Пух! Я был в лесу, собирал цветы для Иа, когда услышал шум и крики. Я побежал посмотреть, что происходит, и увидел, как Тигра и Кролик убегают с чем-то большим и тяжелым. Я попытался их остановить, но они были слишком быстрыми и сильными. Я очень испугался и убежал обратно к Иа.
Детектив Винни-Пух: **Винни-Пух:** Пятачок, ты видел, что именно Тигра и Кролик украли?
**Пятачок:** Нет, я не успел разглядеть. Но это было что-то большое и тяжёлое.
**Винни-Пух**: Ты заметил, в какую сторону они побежали?
**Пятачок**: Да, они бежали туда, где живёт Сова.
**Винни-Пух:** Они тебя заметили? 
**Пятачок**: Думаю, да. Тигра посмотрел на меня, когда мы столкнулись, но не остановился.
**Винни-Пух**: Они что-нибудь говорили, пока убегали?
**Пятачок**: Я слышал, как Кролик смеялся, а Тигра кричал: «Быстрее!».
**Винни-Пух**: Это всё, что ты можешь рассказать? Мо